# Fine Tuning Large Language Model - Model

In this workshop, you will learn how to fine tune the prompts and the LLMs to enhance and improves its response.

In [2]:
# Import libraries
import torch, time
import pandas as pd
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, GenerationConfig, TrainingArguments

from peft import PeftModel, LoraConfig, get_peft_model, TaskType

In [4]:
# TODO: Load and explore the following datasets

dataset_name = "knkarthick/dialogsum"
model_name = "google/flan-t5-small"
model_name = "google/flan-t5-base"

ds = load_dataset(dataset_name)

In [18]:
# TODO: Print a record
print(ds.shape)

idx = 10
print(ds['train'].features)
print(ds['train'][idx])

for k, v in ds['train'][idx].items():
   print(f'k = {k}, v = {v}')


{'train': (12460, 4), 'validation': (500, 4), 'test': (1500, 4)}
{'id': Value('string'), 'dialogue': Value('string'), 'summary': Value('string'), 'topic': Value('string')}
{'id': 'train_10', 'dialogue': "#Person1#: Could you do me a favor?\n#Person2#: Sure. What is it?\n#Person1#: Could you run over to the store? We need a few things.\n#Person2#: All right. What do you want me to get?\n#Person1#: Well, could you pick up some sugar?\n#Person2#: Okay. How much?\n#Person1#: A small bag. I guess we also need a few oranges.\n#Person2#: How many?\n#Person1#: Oh, let's see. . . About six.\n#Person2#: Anything else?\n#Person1#: Yes. We're out of milk.\n#Person2#: Okay. How much do you want me to get? A gallon?\n#Person1#: No. I think a half gallon will be enough.\n#Person2#: Is that all?\n#Person1#: I think so. Have you got all that?\n#Person2#: Yes. That's small bag of sugar, four oranges, and a half gallon of milk.\n#Person1#: Do you have enough money?\n#Person2#: I think so.\n#Person1#: Tha

## Fine tuning the LLM model

In this workshop we will be turning the <code>google/flan-t5-base</code> model.

In [8]:
# Utility function to dump a model's tunable parameters

def print_trainable_model_params(model):
   trainable_model_params = 0
   all_model_params = 0
   for _, param in model.named_parameters():
      all_model_params += param.numel()
      if param.requires_grad:
         trainable_model_params += param.numel()
   return f"Trainable parameters: {trainable_model_params}\nTotal parameters: {all_model_params}\nPercentable of trainable parameters: {100 * trainable_model_params / all_model_params:.2f}%"

In [9]:
# TODO: Load model
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [10]:
# TODO: Print number of trainable parameters
print(print_trainable_model_params(original_model))

Trainable parameters: 247577856
Total parameters: 247577856
Percentable of trainable parameters: 100.00%


In [11]:
print(original_model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

### Preprocess the dialogue dataset

We will train the model to summarize dialogue by creating a dialogue-summary pair for the LLM to process. The dialogue is the training data and the summary is the label. This is supervized learning.

The prompt will be as follows

```
Summarize the following dialogue.\n
\n
Fred: ...\n
Barney: ...\n
\n
Summary:\n
Summary of the conversation between Fred and Barney
```

The prompt and the summary will be tokenized for the LLM

In [12]:
# Utitlity function to prepare the data for training 
# Tokenize function
def tokenize_fn(data):
   start_prompt = 'Summarize the following dialogue.\n\n'
   end_prompt = '\n\nSummary:'
   prompt = [ start_prompt + d + end_prompt for d in data['dialogue'] ]
   summary = data['summary']
   data['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
   data['labels'] = tokenizer(summary, padding="max_length", truncation=True, return_tensors="pt").input_ids
   return data


In [14]:
# TODO: prepare the data for training with the tokenize_fn function
tokenized_dataset = ds.map(tokenize_fn, batched=True)



In [17]:
print(tokenized_dataset['train'][idx])
for k, v in tokenized_dataset['train'][idx].items():
   print(f'k = {k}, v = {v}')

{'id': 'train_10', 'dialogue': "#Person1#: Could you do me a favor?\n#Person2#: Sure. What is it?\n#Person1#: Could you run over to the store? We need a few things.\n#Person2#: All right. What do you want me to get?\n#Person1#: Well, could you pick up some sugar?\n#Person2#: Okay. How much?\n#Person1#: A small bag. I guess we also need a few oranges.\n#Person2#: How many?\n#Person1#: Oh, let's see. . . About six.\n#Person2#: Anything else?\n#Person1#: Yes. We're out of milk.\n#Person2#: Okay. How much do you want me to get? A gallon?\n#Person1#: No. I think a half gallon will be enough.\n#Person2#: Is that all?\n#Person1#: I think so. Have you got all that?\n#Person2#: Yes. That's small bag of sugar, four oranges, and a half gallon of milk.\n#Person1#: Do you have enough money?\n#Person2#: I think so.\n#Person1#: Thanks very much. I appreciate it.", 'summary': '#Person1# asks #Person2# to do a favor. #Person2# agrees and helps buy a small bag of sugar, six oranges, and a half-gallon of

In [21]:
# TODO: Verify prepared data
print(tokenizer.decode(tokenized_dataset['test'][idx]['input_ids'], skip_special_tokens=True))
print(tokenizer.decode(tokenized_dataset['test'][idx]['labels'], skip_special_tokens=True))


Summarize the following dialogue. #Person1#: Happy Birthday, this is for you, Brian. #Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time. #Person1#: Brian, may I have a pleasure to have a dance with you? #Person2#: Ok. #Person1#: This is really wonderful party. #Person2#: Yes, you are always popular with everyone. and you look very pretty today. #Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel. #Person2#: You look great, you are absolutely glowing. #Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday Summary:
#Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.


In [23]:
print(tokenized_dataset['train'].features)

{'id': Value('string'), 'dialogue': Value('string'), 'summary': Value('string'), 'topic': Value('string'), 'input_ids': List(Value('int32')), 'labels': List(Value('int64'))}


In [24]:
# TODO: Remove id, dialogue, summary and topic columns from dataset. We only want input_ids and labels
tokenized_dataset = tokenized_dataset.remove_columns(
   [ 'id', 'dialogue', 'summary', 'topic' ]
)

In [25]:
# TODO: Verify dataset again
print(tokenized_dataset['train'].features)

{'input_ids': List(Value('int32')), 'labels': List(Value('int64'))}


### Tune model with pre-processed dataset

We will use [<code>Trainer</code>](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#api-reference%20][%20transformers.Trainer) to train the original model. The training result will be written out. The trainer will be configure with [<code>TrainingArgument</code>](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments)

In [26]:
# CUDA information

print('CUDA available: ', torch.cuda.is_available())
if torch.cuda.is_available():
   print('B16 supported: ', torch.cuda.is_bf16_supported())
   torch.cuda.set_device(0)
   print('Current device: ', torch.cuda.current_device())
   print('CUDA device name: ', torch.cuda.get_device_name(0))

CUDA available:  False


## Fine tuning the LLM Model with Low-Rank Adaptation (LoRA) / Parameter Efficient Fine Tuning (PEFT)

We will add a LoRA adapter to the LLM (flan-t5-base) and train the adapter. The original LLM will be frozen. The adapter can be combined with the original LLM during inferencing. 

In [27]:
# TODO: Configure LoRA
lora_config = LoraConfig(
   # smaller means be better at what it is currently doing, 
   # larger means ability learn new concepts / 'things'
   r = 16,
   lora_alpha=16,
   target_modules=['q','v'], 
   lora_dropout=0.05,
   bias="none",
   task_type=TaskType.SEQ_2_SEQ_LM
)

In [28]:
# TODO: Add LoRA to the LLM model to be trained
lora_model = get_peft_model(original_model, lora_config)

In [29]:
# TODO: Print number of parameters, compare LoRA to the original model
print(print_trainable_model_params(lora_model))

Trainable parameters: 1769472
Total parameters: 249347328
Percentable of trainable parameters: 0.71%


In [32]:
# TODO: Train model with LoRA
output_dir = f'peft-model-checkpoint-{str(int(time.time()))}'
lora_training_args = TrainingArguments(
   output_dir=output_dir,
   auto_find_batch_size=True, 
   learning_rate=1e-3,
   max_steps=1
)

In [33]:
# TODO: Create trainer and train model
lora_trainer = Trainer(
   model=lora_model,
   args = lora_training_args,
   train_dataset = tokenized_dataset['train'],
   eval_dataset = tokenized_dataset['validation']
)

#lora_trainer.train()

#lora_trainer.model.save_pretrained('chuk/flan-t5-instruct')
#tokenizer.save_pretrained('chuk/flan-65-instruct')

### Use a trained LoRA model

The training will take a few hours and over many iterations.

For the purpose of this workshop we use a save model [intotheverse/peft-dialogue-summary-checkpoint](https://huggingface.co/intotheverse/peft-dialogue-summary-checkpoint).

In [35]:
#TODO: Load the original model and add the pre-trained LoRA adaptation to the model
peft_dialogue_summary_checkpoint = 'intotheverse/peft-dialogue-summary-checkpoint'

lora_base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

lora_model = PeftModel.from_pretrained(
      lora_base_model, 
      peft_dialogue_summary_checkpoint,
      torch_dtype=torch.bfloat16, 
      is_trainable=False)

print(print_trainable_model_params(lora_model))

`torch_dtype` is deprecated! Use `dtype` instead!


Trainable parameters: 0
Total parameters: 251116800
Percentable of trainable parameters: 0.00%


In [36]:
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

## Evaluate LoRA model

In [ ]:
# TODO: Evaluate LoRA model against the original 



In [37]:
# Prepare data for evaluation
dialogues = []
summaries = []
orig_model_summaries = []
lora_model_summaries = []
dataset = ds
config = GenerationConfig(max_new_tokens=200)

for i in range(5):
   print(f'i = {i}')
   d = dataset['test'][i]['dialogue']
   s = dataset['test'][i]['summary']
   prompt = f"Summarize the following conversation.\n\n{d}\n\nSummary:"
   tokenized_prompt = tokenizer(prompt, return_tensors='pt').input_ids
   orig_resp = original_model.generate(input_ids=tokenized_prompt, generation_config=config)
   orig_resp_text = tokenizer.decode(orig_resp[0], skip_special_tokens=True)
   lora_resp = lora_model.generate(input_ids=tokenized_prompt, generation_config=config)
   lora_resp_text = tokenizer.decode(lora_resp[0], skip_special_tokens=True)

   summaries.append(s)
   orig_model_summaries.append(orig_resp_text)
   lora_model_summaries.append(lora_resp_text)

zipped_summaries = list(zip(summaries, orig_model_summaries, lora_model_summaries))
df = pd.DataFrame(zipped_summaries, columns=['label', 'original_model_summary', 'lora_model_summary'])
df

i = 0
i = 1
i = 2
i = 3
i = 4


,label,original_model_summary,lora_model_summary
0,Ms. Dawson helps #Person1# to write a memo to ...,The memo is to be distributed to all employees...,#Person1# asks Ms. Dawson to take a dictation ...
1,In order to prevent employees from wasting tim...,The memo is to be distributed to all employees...,#Person1# asks Ms. Dawson to take a dictation ...
2,Ms. Dawson takes a dictation for #Person1# abo...,The memo is to be distributed to all employees...,#Person1# asks Ms. Dawson to take a dictation ...
3,#Person2# arrives late because of traffic jam....,The traffic jam at the Carrefour intersection ...,#Person2# got stuck in traffic and #Person1# s...
4,#Person2# decides to follow #Person1#'s sugges...,The traffic jam at the Carrefour intersection ...,#Person2# got stuck in traffic and #Person1# s...


### Evaluate models with ROUGE/Bleu metrics

Recall-Oriented Understudy for Gisting Evaluate ([ROUGE](https://pub.aimind.so/unveiling-the-power-of-rouge-metrics-in-nlp-b6d3f96d3363)) is a set of metrics used to evaluate the quality of machine-generated text, such as summaries and translations. ROUGE metrics compare the generated text to a human-written reference and measure the overlap between the two. 

The metrics range between 0 and 1, with higher scores indicating higher similarity between the baseline and generated text.

In [39]:
# TODO: create ROUGE metrics
rouge = evaluate.load('rouge')

In [40]:
# TODO: Evaluate the original model's result
orig_model_results = rouge.compute(
   references=summaries,
   predictions=orig_model_summaries,
   use_aggregator=True,
   use_stemmer=True
)

loral_model_results = rouge.compute(
   references=summaries,
   predictions=lora_model_summaries,
   use_aggregator=True,
   use_stemmer=True
)


In [43]:
print('original:', orig_model_results)

print('------------')

print('LoRA:', loral_model_results)

original: {'rouge1': np.float64(0.17391941391941393), 'rouge2': np.float64(0.03820816864295125), 'rougeL': np.float64(0.13364468864468865), 'rougeLsum': np.float64(0.13364468864468865)}
------------
LoRA: {'rouge1': np.float64(0.34193513803269904), 'rouge2': np.float64(0.1024924201890494), 'rougeL': np.float64(0.2728511771470072), 'rougeLsum': np.float64(0.2728511771470072)}


In [ ]:
# TODO: Evaluate with Bleu metrics
